In [1]:
import pandas as pd

ANO = 2019

In [2]:
t1 = pd.read_csv(f"data/{ANO}/negocios_criados.csv")

t2 = pd.read_csv(f"data/{ANO}/agendamento.csv")

t3 = pd.read_csv(f"data/{ANO}/agendamento_aceito.csv")

t4 = pd.read_csv(f"data/{ANO}/diagnostico.csv")

t5 = pd.read_csv(f"data/{ANO}/proposta.csv")

t6 = pd.read_csv(f"data/{ANO}/cliente.csv")

# t7 = pd.read_excel("data/06 cliente.xlsx", sheet_name='Contagem de Negócios   Cliente ')



In [4]:
t1_t2 = t1.merge(t2, on='Record ID', how='left')
t1_t2.shape

t1_t2 = t1_t2.rename(columns={
    'Nome do negócio_x': 'Nome do negócio',
    'Etapa do negócio_x': 'Etapa do negócio',
    'Valor na moeda da empresa_x': 'Valor na moeda da empresa',
    'Nome do negócio_y':'flag_corresponde_1',
    'Data de fechamento_x': 'Data de fechamento',
    'Pipeline_x': 'Pipeline',
}).assign(flag_corresponde_1 = lambda df_: df_['flag_corresponde_1'].notnull().map({True: "Sim", False: "Nao"}))\
    .drop(columns=['Valor na moeda da empresa_y', 'Etapa do negócio_y'])

In [6]:


t1_t2_t3 = t1_t2.merge(t3, on='Record ID', how='left')\
    .rename(columns={
        'Nome do negócio_x': 'Nome do negócio',
        'Valor na moeda da empresa_x': 'Valor na moeda da empresa',
        'Etapa do negócio_x': 'Etapa do negócio',
        'Data de fechamento_x': 'Data de fechamento',
        'Nome do negócio_y': 'flag_corresponde_2',
        'Pipeline_x': 'Pipeline',
    })\
        .assign(flag_corresponde_2 = lambda df_: df_['flag_corresponde_2'].notnull().map({True: "Sim", False: "Nao"}))\
            .drop(columns=['Valor na moeda da empresa_y', 'Etapa do negócio_y', 'Data de fechamento_y', 'Pipeline_y'])

In [8]:
t1_t2_t3_t4 = t1_t2_t3.merge(t4, on='Record ID', how='left')\
    .rename(columns={
        'Nome do negócio_x': 'Nome do negócio',
        'Valor na moeda da empresa_x': 'Valor na moeda da empresa',
        'Etapa do negócio_x': 'Etapa do negócio',
        'Data de fechamento_x': 'Data de fechamento',
        'Nome do negócio_y': 'flag_corresponde_3',
        'Pipeline_x': 'Pipeline',
    }).assign(flag_corresponde_3 = lambda df_: df_['flag_corresponde_3'].notnull().map({True: "Sim", False: "Nao"}))\
    .drop(columns=['Valor na moeda da empresa_y', 'Etapa do negócio_y', 'Data de fechamento_y', 'Pipeline_y'])

In [9]:
t1_t2_t3_t4_t5 = t1_t2_t3_t4.merge(t5, on='Record ID', how='left')\
    .rename(columns={
        'Nome do negócio_x': 'Nome do negócio',
        'Valor na moeda da empresa_x': 'Valor na moeda da empresa',
        'Etapa do negócio_x': 'Etapa do negócio',
        'Data de fechamento_x': 'Data de fechamento',
        'Nome do negócio_y': 'flag_corresponde_4',
        'Pipeline_x': 'Pipeline',
    }).assign(flag_corresponde_4 = lambda df_: df_['flag_corresponde_4'].notnull().map({True: "Sim", False: "Nao"}))\
    .drop(columns=['Valor na moeda da empresa_y', 'Etapa do negócio_y', 'Data de fechamento_y', 'Pipeline_y'])

In [10]:
t1_t2_t3_t4_t5_t6 = t1_t2_t3_t4_t5.merge(t6, on='Record ID', how='left')\
    .rename(columns={
        'Nome do negócio_x': 'Nome do negócio',
        'Valor na moeda da empresa_x': 'Valor na moeda da empresa',
        'Etapa do negócio_x': 'Etapa do negócio',
        'Data de fechamento_x': 'Data de fechamento',
        'Nome do negócio_y': 'flag_corresponde_5',
        'Pipeline_x': 'Pipeline',
    }).assign(flag_corresponde_5 = lambda df_: df_['flag_corresponde_5'].notnull().map({True: "Sim", False: "Nao"}))\
    .drop(columns=['Valor na moeda da empresa_y', 'Etapa do negócio_y', 'Data de fechamento_y', 'Pipeline_y'])

In [11]:
t1_t2_t3_t4_t5_t6 = t1_t2_t3_t4_t5_t6\
    .assign(**{"Nome do negócio": lambda df_: df_['Nome do negócio'].str.strip()})\
    .rename(
    columns={
        'flag_corresponde_1': 'Agendamentos',
        'flag_corresponde_2': 'Agendamento_Aceito',
        'flag_corresponde_3': 'Diagnostico',
        'flag_corresponde_4': 'Proposta',
        'flag_corresponde_5': 'Cliente'
    }
)

In [13]:
empresas = pd.read_csv("data/{ANO}/empresas_{ANO}.csv")

negocios = pd.read_csv("data/{ANO}/negocios_{ANO}.csv")

negocios = negocios.loc[:, ['Record ID', 'Nome do negócio', 'Valor', 'Valor do contrato anual', 'Motivo de fechamento perdido']]\
.assign(**{'Nome do negócio': lambda df: df['Nome do negócio'].str.upper()})

empresas = empresas.dropna(subset=['Nome da empresa'])
empresas = empresas.loc[:, ['Record ID', 'Nome da empresa', 'Setor']]

empresas = empresas.assign(**{
    'Nome da empresa': lambda df_: df_['Nome da empresa'].str.strip()
})
empresas['Nome da empresa'] = empresas['Nome da empresa'].str.upper()

In [14]:
import fuzzy_pandas as fpd

In [22]:
funil_negocios = t1_t2_t3_t4_t5_t6.merge(negocios, on='Record ID', how='left')\
    .rename(columns={
        'Nome do negócio_x': 'Nome do negócio',
        })\
    .drop(columns=['Nome do negócio_y'])\
    .assign(**{'Nome do negócio': lambda df: df['Nome do negócio'].str.upper().str.split('-').str[0]})


results = fpd.fuzzy_merge(funil_negocios, empresas, left_on='Nome do negócio', right_on='Nome da empresa',
                          join='left-outer',
                          method = 'levenshtein', threshold = 0.70)
    # .merge(empresas.assign(**{'Nome da empresa': lambda df: df['Nome da empresa'].str.upper()}), left_on='Nome do negócio', right_on='Nome da empresa', how='left')\
    # .rename(columns={
    #     'Record ID_x': "Record ID"
    # })\
    # .loc[:, ['Record ID', 'Nome do negócio', 'Data de criação', 'Data de fechamento', 'ultima_etapa', 'Valor', 'Valor do contrato anual', 'Motivo de fechamento perdido', 'Setor']]


print("Found", results.shape)


Found (13, 17)


,Record ID,Nome do negócio,Valor na moeda da empresa,Etapa do negócio,Agendamentos,Data de fechamento,Pipeline,Agendamento_Aceito,Diagnostico,Proposta,Cliente,Valor,Valor do contrato anual,Motivo de fechamento perdido,Record ID,Nome da empresa,Setor
0,3453054574,INTERNATIONAL COMMITTEE OF THE RED CROSS,93070.04,Negócio perdido,Sim,2021-01-12 14:41,Enterprise - Macfor Dealflow,Sim,Sim,Sim,Nao,93070.04,NaN,Fechou com Concorrente,4852527275,INTERNATIONAL COMMITTEE OF THE RED CROSS,Organização cívica e social
1,3362452546,FUNDAÇÃO ESTUDAR,189838.68,Negócio perdido,Sim,2021-01-12 14:33,Enterprise - Macfor Dealflow,Sim,Sim,Sim,Nao,189838.68,NaN,Não era decisor,4766021305,FUNDAÇÃO ESTUDAR,Gestão de organizações sem fins lucrativos
2,3269625040,EUROFARMA,60000.00,Negócio perdido,Sim,2021-02-01 12:27,Enterprise - Macfor Dealflow,Sim,Nao,Nao,Nao,60000.00,NaN,Fechou com Concorrente,3342286545,EUROFARMA,Farmacêutica
3,3142655559,EMBRAMACO,192268.08,Negócio perdido,Sim,2021-02-25 12:40,Enterprise - Macfor Dealflow,Sim,Sim,Sim,Nao,192268.08,NaN,Nós Declinamos,4121901862,EMBRAMACO,Materiais de construção
4,3086522280,EUROFARMA,20964.00,Negócio perdido,Sim,2021-01-12 14:34,Enterprise - Macfor Dealflow,Sim,Sim,Sim,Nao,20964.00,NaN,Não era decisor,3342286545,EUROFARMA,Farmacêutica
5,2459718282,UNIVERSIDADE DE SÃO PAULO,993341.96,Negócio perdido,Nao,NaN,NaN,Nao,Nao,Nao,Nao,993341.96,NaN,Desinteresse/Declinou; Sumiu,4188640137,UNIVERSIDADE DE SÃO PAULO,Ensino superior
6,3565923730,RAÍZEN (SHELL BOX),NaN,Negócio perdido,Nao,NaN,NaN,Nao,Nao,Nao,Nao,NaN,NaN,Maturação,,,
7,3530577142,TINTAS EUCATEX,240000.00,Negócio perdido,Nao,NaN,NaN,Nao,Nao,Nao,Nao,240000.00,NaN,Desinteresse/Declinou,,,
8,3488625670,CARGILL FOOD SERVICE,147831.87,Negócio perdido,Sim,2021-03-18 21:51,Enterprise - Macfor Dealflow,Sim,Sim,Sim,Nao,147831.87,NaN,Desinteresse/Declinou,,,
9,3084963555,CARGILL GENUINE,120000.00,Negócio perdido,Sim,2021-03-16 17:47,Enterprise - Macfor Dealflow,Sim,Sim,Sim,Nao,120000.00,NaN,Sumiu,,,


In [24]:
results.to_excel("data/{ANO}/funil_negocios_{ANO}.xlsx", index=False)